In [1]:
!pip install bpy

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 68.6 MB/s eta 0:00:00


In [3]:
import os
import bpy
import time

# dataset:
# fbx files (main dataset source)
FBX = '/content/drive/MyDrive/AutoCalibr/dataset/fbx/'
# ply (pre-converted to fix artifacts)
PLY_TO_IMPORT = '/content/drive/MyDrive/AutoCalibr/dataset/ply/'

# intermediate folders
INTERMEDIATES = '/content/intermediates/'
CONVERTED_PLY = '/content/intermediates/converted_ply/'
NORMALIZED_PLY = '/content/intermediates/normalized_ply/'

# sometimes used for debug outputting into non-cluttered directory
DIR = '/content/'

In [4]:
#@title { vertical-output: true}

!rm -r sample_data/
!mkdir {INTERMEDIATES} 2>/dev/null
!rm -r {CONVERTED_PLY} 2>/dev/null
!mkdir {CONVERTED_PLY}

# needs to be rotated counterclockwise (left) 90 degrees - could be mostly automated (noted below)
needs_extra_rotation = ["Ace Of Spades", "Cantata-57", "Chroma Rush", "Cloudstrike", "Dead Mans Tale", "Duality", "False Promises", "Fugue 55", "Hawkmoon", "Jack Queen King 3", "Mindbenders Ambition", "No Time To Explain", "Ruinous Effigy", "Seven Seraph Carbine", "Seventh Seraph CQC-12", "Seventh Seraph Officer Revolver", "Seventh Seraph SAW", "Seventh Seraph SI-2", "Seventh Seraph VY-7", "The Fourth Horseman", "Trustee", "Witherhoard"]

for f in os.listdir(FBX):
  if f.endswith('.fbx'):
    # Isolate the name of the .fbx file (without extension)
    name = os.path.splitext(os.path.basename(f))[0]

    # Delete all mesh objects to avoid exporting multiple models into the same file
    bpy.ops.object.select_all(action='DESELECT')
    bpy.ops.object.select_by_type(type='MESH')
    bpy.ops.object.delete()

    # Load in FBX file
    bpy.ops.import_scene.fbx(filepath=os.path.join(FBX, f))

    # Select the object
    obj_object = bpy.context.selected_objects[0]
    bpy.context.view_layer.objects.active = obj_object

    # Decide on redirection
    if name in needs_extra_rotation:
      forwards_dir = 'Z'
      # Remove the file from the list as it's been accounted for
      needs_extra_rotation.remove(name)
    else:
      forwards_dir = '-X'

    # Export object to PLY
    bpy.ops.export_mesh.ply(filepath=os.path.join(CONVERTED_PLY, f.replace('.fbx', '.ply')), use_ascii=True, use_mesh_modifiers=True, use_normals=False, use_uv_coords=False, use_colors=False, axis_forward=forwards_dir, axis_up='Y')

    print()

# Ensure that any models that were supposed to receive extra rotation were hit
if len(needs_extra_rotation) > 0:
  print(f"The following models were not hit and may need manual inspection: {needs_extra_rotation}")


FBX version: 7400
Export completed '/content/intermediates/converted_ply/Coldheart.ply' in 0.446

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Vigilance Wing.ply' in 0.841

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Riskrunner.ply' in 0.173

FBX version: 7400
Export completed "/content/intermediates/converted_ply/Skyburner's oath.ply" in 0.049

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Prometheus Lens.ply' in 0.239

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Anarchy.ply' in 0.541

FBX version: 7400
Export completed '/content/intermediates/converted_ply/The Last Word.ply' in 0.026

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Lord of Wolves.ply' in 0.069

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Sleeper Simulant.ply' in 0.060

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Huckleberry.

In [5]:
class Vertex:
  def __init__(self, x, y, z):
    self.x = x
    self.y = y
    self.z = z

  def scale(self, scale_x, scale_y, scale_z):
    self.x *= scale_x
    self.y *= scale_y
    self.z *= scale_z

  def translate(self, offset_x, offset_y, offset_z):
    self.x += offset_x
    self.y += offset_y
    self.z += offset_z


class Face:
  def __init__(self, vertices):
    self.vertices = vertices


class PlyObject:
  def __init__(self, name, vertices, faces):
    self.name = name
    self.vertices = vertices
    self.faces = faces


  def save_file(self, filename):
    with open(filename, "w") as file:
      file.write("ply\n")
      file.write("format ascii 1.0\n")
      file.write("comment Created by PlyObject class\n")
      file.write(f"element vertex {len(self.vertices)}\n")
      file.write("property float x\n")
      file.write("property float y\n")
      file.write("property float z\n")
      file.write(f"element face {len(self.faces)}\n")
      file.write("property list uchar uint vertex_indices\n")
      file.write("end_header\n")

      for vertex in self.vertices:
        file.write(f"{vertex.x} {vertex.y} {vertex.z}\n")

      for face in self.faces:
        formatted_vertices = ' '.join(str(v) for v in face.vertices)
        number_of_vertices = len(face.vertices)
        file.write(f"{number_of_vertices} {formatted_vertices}\n")


  def scale(self, scale_x, scale_y, scale_z):
    for vertex in self.vertices:
      vertex.scale(scale_x, scale_y, scale_z)


  def translate(self, offset_x, offset_y, offset_z):
    for vertex in self.vertices:
      vertex.translate(offset_x, offset_y, offset_z)


  def get_max_values(self):
    max_values = {'x': None, 'y': None, 'z': None}

    for vertex in self.vertices:
      if max_values['x'] is None or vertex.x > max_values['x']:
        max_values['x'] = vertex.x
      if max_values['y'] is None or vertex.y > max_values['y']:
        max_values['y'] = vertex.y
      if max_values['z'] is None or vertex.z > max_values['z']:
        max_values['z'] = vertex.z

    return max_values


  def get_min_values(self):
    min_values = {'x': None, 'y': None, 'z': None}

    for vertex in self.vertices:
      if min_values['x'] is None or vertex.x < min_values['x']:
        min_values['x'] = vertex.x
      if min_values['y'] is None or vertex.y < min_values['y']:
        min_values['y'] = vertex.y
      if min_values['z'] is None or vertex.z < min_values['z']:
        min_values['z'] = vertex.z

    return min_values


  def normalize_scale(self):
    x_coordinates = [vertex.x for vertex in self.vertices]
    y_coordinates = [vertex.y for vertex in self.vertices]
    z_coordinates = [vertex.z for vertex in self.vertices]

    max_distance = max(x_coordinates + y_coordinates + z_coordinates)
    min_distance = min(x_coordinates + y_coordinates + z_coordinates)
    normalization_range = max_distance - min_distance

    if normalization_range == 0:
      raise ValueError("Normalization range cannot be zero")

    for vertex in self.vertices:
      vertex.x = 2 * (vertex.x - min_distance) / normalization_range - 1
      vertex.y = 2 * (vertex.y - min_distance) / normalization_range - 1
      vertex.z = 2 * (vertex.z - min_distance) / normalization_range - 1


  def delete_plane(self):
    # Note: only call if the object has a plane artifact!
    max_x_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    min_x_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    max_z_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].z)
    min_z_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].z)

    indices_to_remove = set([max_x_index, min_x_index, max_z_index, min_z_index])

    for f, face in enumerate(self.faces):
      if set(face.vertices).intersection(indices_to_remove) == indices_to_remove:
        del self.faces[f]
        break
    else:
      raise ValueError("Face with all vertices not found")

    for index in sorted(indices_to_remove, reverse = True):
      del self.vertices[index]


  def stretch_to_max(self):
    # note: update scale globals before calling
    self.translate(global_stretch_offset_x, global_stretch_offset_y, global_stretch_offset_z)
    self.scale(global_stretch_scale_x, global_stretch_scale_y, global_stretch_scale_z)


  def revert_stretch(self):
    self.scale(1/global_stretch_scale_x, 1/global_stretch_scale_y, 1/global_stretch_scale_z)
    self.translate(-global_stretch_offset_x, -global_stretch_offset_y, -global_stretch_offset_z)


  def dimension_range(self):
    # Initialize the min and max values with the first vertex coordinates
    xmin = xmax = obj.vertices[0].x
    ymin = ymax = obj.vertices[0].y
    zmin = zmax = obj.vertices[0].z

    # Iterate over all vertices and update the min and max values
    for vertex in obj.vertices:
      xmin = min(xmin, vertex.x)
      xmax = max(xmax, vertex.x)
      ymin = min(ymin, vertex.y)
      ymax = max(ymax, vertex.y)
      zmin = min(zmin, vertex.z)
      zmax = max(zmax, vertex.z)

    xrang = xmax - xmin
    yrang = ymax - ymin
    zrang = zmax - zmin

    return {'x':xrang, 'y':yrang, 'z':zrang}


  @classmethod
  def from_file(cls, filepath):
    with open(filepath, 'r') as f:
      if next(f).strip() != "ply":
        raise ValueError("The file being read is not a PLY file.")

      for _ in range(2):
        next(f)

      n_vertices = int(next(f).split()[-1])

      for _ in range(3):
        next(f)

      n_faces = int(next(f).split()[-1])

      for _ in range(2):
        next(f)

      vertices = []
      for _ in range(n_vertices):
        x, y, z = map(float, next(f).split())
        vertices.append(Vertex(x, y, z))

      faces = []
      for _ in range(n_faces):
        face_vertices = list(map(int, next(f).split()[1:]))
        faces.append(Face(face_vertices))

    name = os.path.splitext(os.path.basename(filepath))[0]

    return cls(name, vertices, faces)

In [6]:
#@title { vertical-output: true}

ply_objs = []

for f in os.listdir(CONVERTED_PLY):
  if f.endswith('.ply'):
    start_time = time.time()
    ply_obj = PlyObject.from_file(filepath=os.path.join(CONVERTED_PLY, f))
    ply_objs.append(ply_obj)
    elapsed_time = time.time() - start_time

    print(f'Processed into object: {ply_obj.name} (took {elapsed_time:.2f} s)')
    print(f'\nVertice count: {len(ply_obj.vertices)}')
    print(f'Face count: {len(ply_obj.faces)}')
    print('-' * 50)


# Import pre-converted .ply files
for f in os.listdir(PLY_TO_IMPORT):
  if f.endswith('.ply'):
    start_time = time.time()
    ply_obj = PlyObject.from_file(filepath=os.path.join(PLY_TO_IMPORT, f))
    ply_objs.append(ply_obj)
    elapsed_time = time.time() - start_time

    print(f'Processed object: {ply_obj.name} (took {elapsed_time:.2f} s) (manually converted ply)')
    print(f'\nVertice count: {len(ply_obj.vertices)}')
    print(f'Face count: {len(ply_obj.faces)}')
    print('-' * 50)

Processed into object: Transfiguration (took 0.02 s)

Vertice count: 6633
Face count: 7868
--------------------------------------------------
Processed into object: Ruinous Effigy (took 0.09 s)

Vertice count: 15998
Face count: 19518
--------------------------------------------------
Processed into object: Cloudstrike (took 0.22 s)

Vertice count: 30152
Face count: 30020
--------------------------------------------------
Processed into object: 1000 Voices (took 0.14 s)

Vertice count: 22115
Face count: 32382
--------------------------------------------------
Processed into object: Dead Mans Tale (took 0.15 s)

Vertice count: 24883
Face count: 29735
--------------------------------------------------
Processed into object: Jade Jester (took 0.01 s)

Vertice count: 5949
Face count: 6295
--------------------------------------------------
Processed into object: Silicon Neuroma (took 0.14 s)

Vertice count: 19480
Face count: 20925
--------------------------------------------------
Processed 

In [7]:
#@title { vertical-output: true}

# has a giant rectangular plane originally used as a background, will need to be filtered out
has_plane_artifact = ['Abbadon', 'Blind Perdition', 'Ex Machina', 'Komodo-4FR', 'Nova Mortis', 'Trespasser', 'Vestian Dynasty', 'Vouchsafe']

# Remove plane artifact from manually specified objects

# Could also iterate over all objects and use exceptions,
# but would need to give delete_plane function stricter pre-deletion checking
for obj in ply_objs:
  if obj.name in has_plane_artifact:
    has_plane_artifact.remove(obj.name)

    obj_range = obj.dimension_range()

    # Print the absolute maximum and minimum values
    print(f"Object: {obj.name}")
    print("\nDimension Range:")
    print('X:', obj_range['x'])
    print('Y:', obj_range['y'])
    print('Z:', obj_range['z'])

    start_time = time.time()
    obj.delete_plane()
    elapsed_time = time.time() - start_time

    obj_range = obj.dimension_range()

    # Print the absolute maximum and minimum values
    print(f"\nDimension Range (plane deleted in {elapsed_time:.2f} s):")
    print('X:', obj_range['x'])
    print('Y:', obj_range['y'])
    print('Z:', obj_range['z'])

    print('-' * 50)


# Ensure that any models that were supposed to receive extra rotation were hit
if len(has_plane_artifact) > 0:
  print(f"The following models were not hit and may need manual inspection: {has_plane_artifact}")

Object: Vestian Dynasty

Dimension Range:
X: 1.831408
Y: 0.44448
Z: 3.43575

Dimension Range (plane deleted in 0.01 s):
X: 0.9657739999999999
Y: 0.44448
Z: 2.175607
--------------------------------------------------
Object: Nova Mortis

Dimension Range:
X: 1.831408
Y: 0.44357599999999997
Z: 3.43575

Dimension Range (plane deleted in 0.01 s):
X: 1.031532
Y: 0.44357599999999997
Z: 2.466508
--------------------------------------------------
Object: Trespasser

Dimension Range:
X: 1.831408
Y: 0.49324199999999996
Z: 3.43575

Dimension Range (plane deleted in 0.00 s):
X: 0.953549
Y: 0.49324199999999996
Z: 2.055912
--------------------------------------------------
Object: Ex Machina

Dimension Range:
X: 1.831408
Y: 0.37507999999999997
Z: 3.43575

Dimension Range (plane deleted in 0.02 s):
X: 0.965509
Y: 0.37507999999999997
Z: 2.319017
--------------------------------------------------
Object: Blind Perdition

Dimension Range:
X: 1.831408
Y: 0.47424900000000003
Z: 3.43575

Dimension Range (pl

In [8]:
#@title { vertical-output: true}

for obj in ply_objs:
  min_val = float('inf')
  max_val = float('-inf')

  for v in obj.vertices:
    min_val = min(min_val, v.x, v.y, v.z)
    max_val = max(max_val, v.x, v.y, v.z)

  print(f"Object: {obj.name}")
  print("\nAbsolute Extrema in any Dimension:")
  print(f"Overall minimum: {min_val}")
  print(f"Overall maximum: {max_val}")

  obj.normalize_scale()

  min_val = float('inf')
  max_val = float('-inf')

  for v in obj.vertices:
    min_val = min(min_val, v.x, v.y, v.z)
    max_val = max(max_val, v.x, v.y, v.z)

  print("\nAbsolute Extrema in any Dimension (Normalized size):")
  print(f"Overall minimum: {min_val}")
  print(f"Overall maximum: {max_val}")
  print('-' * 50)

Object: Transfiguration

Absolute Extrema in any Dimension:
Overall minimum: -0.791397
Overall maximum: 0.315061

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Ruinous Effigy

Absolute Extrema in any Dimension:
Overall minimum: -0.791955
Overall maximum: 0.268863

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Cloudstrike

Absolute Extrema in any Dimension:
Overall minimum: -1.191967
Overall maximum: 0.297699

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: 1000 Voices

Absolute Extrema in any Dimension:
Overall minimum: -0.620856
Overall maximum: 0.225458

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
----------------

In [9]:
#@title { vertical-output: true}
min_x = min_y = min_z = float('inf')
max_x = max_y = max_z = float('-inf')

for obj in ply_objs:
  for vertex in obj.vertices:
    min_x = min(min_x, vertex.x)
    min_y = min(min_y, vertex.y)
    min_z = min(min_z, vertex.z)
    max_x = max(max_x, vertex.x)
    max_y = max(max_y, vertex.y)
    max_z = max(max_z, vertex.z)

scale_x, scale_y, scale_z = 2/(max_x - min_x), 2/(max_y - min_y), 2/(max_z - min_z)
offset_x, offset_y, offset_z = - (max_x + min_x) / 2, - (max_y + min_y) / 2, - (max_z + min_z) / 2

print(f"Dimensional minimua:")
print(f"X: {min_x}")
print(f"Y: {min_y}")
print(f"Z: {min_z}")

print(f"\nDimensional maxima:")
print(f"X: {max_x}")
print(f"Y: {max_y}")
print(f"Z: {max_z}\n")

print('-' * 50)

print(f"\nDerived Stretch Value:")
print(f"X: {scale_x}")
print(f"Y: {scale_y}")
print(f"Z: {scale_z}")

print(f"\nDerived Offset Value:")
print(f"X: {offset_x}")
print(f"Y: {offset_y}")
print(f"Z: {offset_z}")


Dimensional minimua:
X: -1.0
Y: -0.5837061471688103
Z: -1.0

Dimensional maxima:
X: 0.7606252674271599
Y: 1.0
Z: 1.0

--------------------------------------------------

Derived Stretch Value:
X: 1.1359600688468159
Y: 1.262860539864291
Z: 1.0

Derived Offset Value:
X: 0.11968736628642007
Y: -0.20814692641559485
Z: -0.0


In [10]:
#@title { vertical-output: true}

# used for checking if I missed any rotation overrides
count_smallest = {'x': 0, 'y': 0, 'z': 0}
count_middle = {'x': 0, 'y': 0, 'z': 0}
count_largest = {'x': 0, 'y': 0, 'z': 0}

# Keep track of absolute min and max for each dimension
track_min = {'x': float('inf'), 'y': float('inf'), 'z': float('inf')}
track_max = {'x': float('-inf'), 'y': float('-inf'), 'z': float('-inf')}

!rm -r {NORMALIZED_PLY} 2>/dev/null
!mkdir {NORMALIZED_PLY}

global_stretch_scale_x = scale_x
global_stretch_scale_y = scale_y
global_stretch_scale_z = scale_z
global_stretch_offset_x = offset_x
global_stretch_offset_y = offset_y
global_stretch_offset_z = offset_z

for obj in ply_objs:
  obj_range = obj.dimension_range()

  print(f"Object: {obj.name}")
  print("\nDimension Range:")
  print('X:', obj_range['x'])
  print('Y:', obj_range['y'])
  print('Z:', obj_range['z'])

  start_time = time.time()
  obj.stretch_to_max()
  elapsed_time = time.time() - start_time

  obj_range = obj.dimension_range()

  # Print the absolute maximum and minimum values
  print(f"\nDimension Range (stretched in {elapsed_time:.2f} s):")
  print('X:', obj_range['x'])
  print('Y:', obj_range['y'])
  print('Z:', obj_range['z'])
  print('-' * 50)

  # counting the podium placings of ranges for every dimension
  sorted_keys = sorted(obj_range, key=obj_range.get)
  smallest_key = sorted_keys[0]
  middle_key = sorted_keys[1]
  largest_key = sorted_keys[2]

  count_smallest[smallest_key] += 1
  count_middle[middle_key] += 1
  count_largest[largest_key] += 1

  # Compare and store the min and max for each dimension

  max_values = obj.get_max_values()
  min_values = obj.get_min_values()

  for dimension in obj_range.keys():
    track_min[dimension] = min(track_min[dimension], min_values[dimension])
    track_max[dimension] = max(track_max[dimension], max_values[dimension])

Object: Transfiguration

Dimension Range:
X: 0.12668533283685446
Y: 0.551764278445273
Z: 2.0

Dimension Range (stretched in 0.00 s):
X: 0.1439094794112349
Y: 0.6968013345552284
Z: 2.0
--------------------------------------------------
Object: Ruinous Effigy

Dimension Range:
X: 0.19434059376820545
Y: 0.5658501269774832
Z: 2.0

Dimension Range (stretched in 0.01 s):
X: 0.2207631542766617
Y: 0.7145897968370619
Z: 2.0
--------------------------------------------------
Object: Cloudstrike

Dimension Range:
X: 0.11114840507872237
Y: 0.3806061224462396
Z: 2.0

Dimension Range (stretched in 0.02 s):
X: 0.1262601498854392
Y: 0.4806524532681126
Z: 2.0
--------------------------------------------------
Object: 1000 Voices

Dimension Range:
X: 0.3734689488771308
Y: 0.8419380986253331
Z: 1.9906441344465529

Dimension Range (stretched in 0.02 s):
X: 0.4242458128786134
Y: 1.0632504017623028
Z: 1.9906441344465529
--------------------------------------------------
Object: Dead Mans Tale

Dimension Ran

In [11]:
#@title { vertical-output: true}

# manually rotated objects so this doesn't matter, but I could use this info to auto-rotate objects if using a large dataset
print("\nTimes dimension had largest dimensional range:")
print(f"X: {count_largest['x']}")
print(f"Y: {count_largest['y']}")
print(f"Z: {count_largest['z']}")

print("\nTimes dimension had middle dimensional range:")
print(f"X: {count_middle['x']}")
print(f"Y: {count_middle['y']}")
print(f"Z: {count_middle['z']}")

print("\nTimes dimension had smallest dimensional range:")
print(f"X: {count_smallest['x']}")
print(f"Y: {count_smallest['y']}")
print(f"Z: {count_smallest['z']}\n")


print('-' * 50)

# shows that our stretch worked
print("\nDimensional Minima:")
print('X:', track_min['x'])
print('Y:', track_min['y'])
print('Z:', track_min['z'])

print("\nDimensional Maxima:")
print('X:', track_max['x'])
print('Y:', track_max['y'])
print('Z:', track_max['z'])


Times dimension had largest dimensional range:
X: 0
Y: 0
Z: 149

Times dimension had middle dimensional range:
X: 9
Y: 140
Z: 0

Times dimension had smallest dimensional range:
X: 140
Y: 9
Z: 0

--------------------------------------------------

Dimensional Minima:
X: -1.0
Y: -1.0
Z: -1.0

Dimensional Maxima:
X: 1.0
Y: 1.0
Z: 1.0


In [12]:
#@title { vertical-output: true}

!rm -r {NORMALIZED_PLY} 2>/dev/null
!mkdir {NORMALIZED_PLY}

for obj in ply_objs:
  start_time = time.time()
  obj.revert_stretch()
  obj.save_file(NORMALIZED_PLY + obj.name + '.ply')
  elapsed_time = time.time() - start_time

  print(f'Exported object: {obj.name} (took {elapsed_time:.2f} seconds)\n')

Exported object: Transfiguration (took 0.03 seconds)

Exported object: Ruinous Effigy (took 0.06 seconds)

Exported object: Cloudstrike (took 0.09 seconds)

Exported object: 1000 Voices (took 0.09 seconds)

Exported object: Dead Mans Tale (took 0.09 seconds)

Exported object: Jade Jester (took 0.03 seconds)

Exported object: Silicon Neuroma (took 0.06 seconds)

Exported object: Prometheus Lens (took 0.07 seconds)

Exported object: Lumina (took 0.06 seconds)

Exported object: Bastion (took 0.08 seconds)

Exported object: Redrix's Claymore (took 0.04 seconds)

Exported object: Pluperfect (took 0.06 seconds)

Exported object: Ikelos_SG_V1.0.1 (took 0.08 seconds)

Exported object: Blasphemer (took 0.05 seconds)

Exported object: Garden Progeny (took 0.09 seconds)

Exported object: Austringer (took 0.04 seconds)

Exported object: Ikelos SR v1.0.1 (took 0.09 seconds)

Exported object: Vestian Dynasty (took 0.04 seconds)

Exported object: Red Death (took 0.04 seconds)

Exported object: Crimso